# MusicLab Notebook for Data Analysis - RPPW
Edited by Dana (June 10-, 2021) from the file used for the lab presentation to analyze data for the RPPW Conference.
MusicLab5: Lockdown Rave took place on July 10th therefore all data needs to have a timestamp indicating that date.  
Event start:  
UTC 13:30  
15:30 in Oslo 
19:00 in India  
22:30 in Japan

## Performance times in UTC
Renick Bell performance: ~13:54-14:25
Khoparzi performance: ~14:34-15:04
Performance times were extracted by examining Renick's computer time when he started and ended and then examining how much time was between Renick and Khoparzi's performances.

## Baselines & IDing start of Music
Before the performance we measured 2 baselines. Baseline 1: 10-seconds flat surface. Baseline 2: 10-seconds still body. 
Participants were instructed to place their phones on a flat surface, to press start recording and then to shake their phones when they heard the music and then to place them on their bodies. 

In checking the duration, and in particular the start time of each recording, I learned that the longest file contains all motion information from that participant. Therefore, I simply need to locate the file with the longest duration and load that as the file for each participant.


Solveig = participant 3  
Renick = paricipant 10  
Dana = participant 11  
Khoparzi's data was lost (app malfunctioned)

## 1. Importing libraries

In [ ]:
import os, fnmatch
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline
# some plots

In [ ]:
matplotlib.rc('xtick', labelsize=14)     
matplotlib.rc('ytick', labelsize=14)
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['legend.fontsize'] = 20
matplotlib.rcParams['figure.titlesize'] = 16

## 2. Directory management

In [ ]:
path = '/Users/danasw/Documents/PhD/Research/MusicLab_Lockdown/Files_for_Research_Drive/Raw_Data/Movement_Data/'
datasets = ['Pt-0'+str(nr) for nr in range(1,10)]
datasets = datasets+['Pt-'+str(nr) for nr in range(10,20)]

In [ ]:
def ffind(pattern, path):
    result=[]
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

## 3. Reading & writing data into Pandas Dataframe
#### Create dictionary of dataframes
Extract the file with the longest duration as the file that will be examined for each participant.

TODO: update the script to also extract the correct geolocation file.

In [ ]:
def GetLongestFile(dictionary):
    return max((len(v), k) for k,v in dictionary.items())   

In [ ]:
df_motion_files={}
df_geo_dict={}
df_geo_files={}
df_motion_dict = {}
for datset in range(0,19):
    pt = datset+1
    participant_dict = {}
    file_dict = {}
    n = 0
    for dat in ffind('*.csv', os.path.join(path, datasets[datset])):
        if 'Motion' in dat:
            print('Dataset #'+ str(pt) + ' IMU √')
            df_motion = pd.read_csv(dat, index_col=1)
            n+=1
            participant_dict[n] = df_motion
            file_dict[n] = os.path.relpath(dat)
        elif 'geo' in dat:
            print('Dataset #'+ str(pt) + ' Geo √')
            df_geo = pd.read_csv(dat, index_col=1)
            df_geo_dict[pt] = df_geo
            df_geo_files[pt] = os.path.relpath(dat)
    # find the participants' longest motion file 
    (v, k) = GetLongestFile(participant_dict)
    if v > 7200: # if file is longer than 2-minutes
        longest = participant_dict[k]
        # convert index to human readable datetime
        longest.loc[:,"datetime"] = pd.to_datetime(longest.timestamp, unit='ms')
        longest = longest.set_index(['datetime'])
        df_motion_dict[pt] = longest
        df_motion_files[pt] = file_dict[k]
    

## Renick Bell's Motion
Update participant number.  
Solveig = participant 3  
Renick = paricipant 10  
Dana = participant 11  
Khoparzi's data was lost *

In [ ]:
pt_num = 10
df_motion = df_motion_dict[pt_num]

### Identify when Renick began the motion recording

In [ ]:
print(df_motion.sort_index().first_valid_index())

![](app-imu-small.png)

In [ ]:
# first plot entire motion
df_motion.iloc[:, 1:4].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green'])

## Extract Renick's performance
Renick = paricipant 10  

#### Estimate average sampling frequency 

In [ ]:
freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')

### Renick Performance Time
We know when Renick started and stopped performing thanks to his computer's time stamp which is in the bottom right corner of his screen in the YouTube video.
13:54-14:25


In [ ]:
## Extract performance
df_motion = df_motion.loc['2020-07-10 13:54' : '2020-07-10 14:25.5' , :]

In [ ]:
df_motion.loc[:, "xaxis_s"] = np.linspace(0, len(df_motion)/freq, len(df_motion)) #seconds
df_motion.loc[:, "xaxis_min"] = np.linspace(0, (len(df_motion)/freq)/60, len(df_motion)) # minutes

In [ ]:
df_motion.plot(x = "xaxis_min", y = ["x","y", "z"],figsize=(24,3), color = ['tab:orange','tab:blue','tab:green'])
plt.xlabel("time (min)")
plt.xticks(np.arange(0, (len(df_motion)/freq)/60, step=5))

## Extract excerpts of audience's movement from Renick's performance
Recall that participants viewing in real-time would have viewed the stream 15-seconds after Renick performed it. It is also possible some audience were not watching in realtime so be sure to check that there is shaking of the phone near the time you would expect it. If not, you may need to adjust the data that is extracted for that participant. 

Another way: I need to manually or automatically detect the beginning of the shakes.
Finn suggested using second and third order differences to detect the beginning of the shake.
Another way is to manually go throguh each plot and ID by clicking using matplotlib ginput function. 

In [ ]:
## Remove participants who have no data during renick's performance (pt 6, 7, 16)
renick_audience = {}
for k,v in df_motion_dict.items():
    df_motion = df_motion_dict[k]
    df_motion_r = df_motion.sort_index().loc['2020-07-10 13:54' : '2020-07-10 14:25.5', :] # 13:54-14:25.5
    if k != 10 and len(df_motion_r)>0: # remove any dataframes not containing data during Renick's performance and Renick's performance
        renick_audience[k]=df_motion    

#  Identify the shaking in the data
The shaking could not have occurred before Renick started his performance. Renick's performance was between 13:54 - 14:27 therefore there is no shaking before 13:54
Participant 14: doesn't have much data from renick's performance
Shaking signal seems to be the largest signal across all participants

In [ ]:
# Extract first minute after performance start
df_motion = renick_audience[3]
df_motion.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"
start_idx =  df_motion.sort_index().first_valid_index()
start_time = pd.to_datetime('2020-07-10 13:54')# look at 13:54 because nothing before this could be the shake
time_change = pd.Timedelta("2 minutes")
end_time = start_time + time_change
df_motion_one_min = df_motion.sort_index().loc[start_time : end_time, :]
df_motion_one_min.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"

## Make the same plots in matplotlib

In [ ]:
# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(df_motion_one_min.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')

# create x-axis
time = np.linspace(0, (len(df_motion_one_min)/freq)/60, len(df_motion_one_min)) # minutes

fig, ax = plt.subplots( figsize=(24,3)) # nrows=3, ncols=1,

ax.plot(df_motion_one_min.index.values, df_motion_one_min['x'])
ax.plot(df_motion_one_min.index.values, df_motion_one_min['y'])
ax.plot(df_motion_one_min.index.values, df_motion_one_min['z'])

shake_start = plt.ginput(1)

plt.show()

It is unclear what the number in the x-axis represents when we are using the index values, therefore slicing might be easeir if we plot with the timestamp instead.

In [ ]:
fig, ax = plt.subplots( figsize=(24,3)) # nrows=3, ncols=1,

ax.plot(df_motion_one_min['timestamp'], df_motion_one_min['x'])
ax.plot(df_motion_one_min['timestamp'], df_motion_one_min['y'])
ax.plot(df_motion_one_min['timestamp'], df_motion_one_min['z'])

shake_start = plt.ginput(1)

plt.show()

In [ ]:
start = pd.to_datetime(shake_start[0][0], unit='s')


## Second order difference

In [ ]:
df_motion_diff_two = df_motion_one_min.loc[:, "x":"z"].diff(2)
df_motion_diff_two.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"

## Third order difference

In [ ]:
df_motion_diff_three = df_motion_one_min.loc[:, "x":"z"].diff(3)
df_motion_diff_three.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"

### Plot full data and then the excerpt

In [ ]:
for k,v in renick_audience.items():
    excerpt = renick_audience[k]
    # plot full 
    full = df_motion_dict[k]
    df_motion_dict[k].loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"
    plt.title(label = "Pt-"+str(k))
    # to compare with excerpt
    excerpt.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"

## Extract excerpts of audience motion from Khoparzi performance

In [ ]:
## Remove participants who have no data during renick's performance (pt 1,9,10,11,18)
khoparzi_audience = {}
for k,v in df_motion_dict.items():
    df_motion = df_motion_dict[k]
    df_motion = df_motion.sort_index().loc['2020-07-10 14:34' : '2020-07-10 15:04', :] # Real times: '2020-07-10 14:36' : '2020-07-10 15:04'; or 14:34-15:04 
    if k != 3 and len(df_motion)>100: # remove any dataframes not containing more than 100 samples and participant 3 who does not seem to have motion after the shake
        khoparzi_audience[k]=df_motion    

In [ ]:
for k,v in khoparzi_audience.items():
    excerpt = khoparzi_audience[k]
    # plot full 
    full = df_motion_dict[k]
    df_motion_dict[k].loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"
    plt.title(label = "Pt-"+str(k))
    # to compare with excerpt
    excerpt.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"

### Plot 1-min of data from one participant
Pt-4, 13, and 14 look like they have movement. Let's check out some 1-minute intervals of theirs

In [ ]:
k = 4
dance_excerpt = khoparzi_audience[k]
dance_1min = dance_excerpt.sort_index().loc['2020-07-10 14:45' : '2020-07-10 14:46', :] # Real times: '2020-07-10 14:36' : '2020-07-10 15:04'
dance_1min.loc[:, "x":"z"].plot(figsize=(24,3), color = ['tab:orange','tab:blue','tab:green']) # if want minutes use x = "xaxis_min"
plt.title(label = "Pt-"+str(k))

## 4. Pre-processing the data
### 4.1. Filtering
##### e.g. 4th order, zero-phase IIR lowpass or bandpass filter

In [ ]:
from scipy.signal import butter, lfilter

def butter_filt(data, lowcut, highcut, fs, order=4, btype='band'):
    nyq = fs / 2
    b, a = butter(order, [lowcut/nyq, highcut/nyq], btype=btype)
    y = lfilter(b, a, data)
    return y

## Define filter parameters
Finn suggested 0.5-5Hz because 10 Hz is a really high frequency for human motion

In [ ]:
lowcut = 0.5
highcut = 5

In [ ]:
renick_filtered = {}
for k,v in renick_audience.items():
    df_motion = renick_audience[k]
    columns = list(df_motion.iloc[:, 1:4])
    filtered=[]
    freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
    for col in columns:
        filt = butter_filt(df_motion[col], lowcut, highcut, fs=freq, order=4)
        filtered.append(filt)
    renick_filtered[k] = filtered

In [ ]:
khoparzi_filtered = {}
for k,v in khoparzi_audience.items():
    df_motion = khoparzi_audience[k]
    columns = list(df_motion.iloc[:, 1:4])
    filtered=[]
    freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
    for col in columns:
        filt = butter_filt(df_motion[col], lowcut, highcut, fs=freq, order=4)
        filtered.append(filt)
    khoparzi_filtered[k] = filtered

### 4.2. Normalization

In [ ]:
def normalize(y, min_val=0):
    max_value = max(y)
    min_value = min(y)
    k = []
    for i in range(0, len(y)):
        if min_val == 0:
            k.append((y[i] - min_value) / (max_value - min_value))
        elif min_val == -1:
            k.append( 2*(y[i] - min_value) / (max_value - min_value)-1 )
    return np.array(k)

In [ ]:
renick_normalized = {}
for k,v in renick_filtered.items():
    filtnorm=[]
    filtered = renick_filtered[k]
    for ax in filtered:
        norm = normalize(ax, min_val=-1)
        filtnorm.append(norm)
    renick_normalized[k] = filtnorm

In [ ]:
khoparzi_normalized = {}
for k,v in khoparzi_filtered.items():
    filtnorm=[]
    filtered = khoparzi_filtered[k]
    for ax in filtered:
        norm = normalize(ax, min_val=-1)
        filtnorm.append(norm)
    khoparzi_normalized[k] = filtnorm

## Visualize the filtered data

In [ ]:
pt = 1
df_motion = renick_audience[pt]
filt = renick_filtered[pt]

# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')
# create x-axis
time = np.linspace(0, (len(df_motion)/freq)/60, len(df_motion)) # minutes

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))
labels = ['X', 'Y', 'Z']
colors = ['tab:orange','tab:blue','tab:green']

for anr in range(3):
    ax[anr].plot(time, filt[anr], label=labels[anr], linewidth=0.5, color=colors[anr], alpha=0.5)
    ax[anr].legend(loc='lower right', fontsize=14)
ax[0].set_title('X - Lateral - side to side')
ax[1].set_title('Y - Vertical - up down')
ax[2].set_title('Z - Frontal - forwards backwards')

fig.subplots_adjust(hspace=0.5)

## Visualize the normalized data

In [ ]:
pt = 1
df_motion = renick_audience[pt]
filtnorm = renick_normalized[pt]
# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')
# create x-axis
time = np.linspace(0, (len(df_motion)/freq)/60, len(df_motion)) # minutes

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))
labels = ['X', 'Y', 'Z']
colors = ['tab:orange','tab:blue','tab:green']

for anr in range(3):
    ax[anr].plot(time, filtnorm[anr], label=labels[anr], linewidth=0.5, color=colors[anr], alpha=0.5)
    ax[anr].legend(loc='lower right', fontsize=14)
ax[0].set_title('X - Lateral - side to side')
ax[1].set_title('Y - Vertical - up down')
ax[2].set_title('Z - Frontal - forwards backwards')

fig.subplots_adjust(hspace=0.5)

In [ ]:
pt = 13
df_motion = khoparzi_audience[pt]
filtnorm = khoparzi_normalized[pt]
# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')
# create x-axis
time = np.linspace(0, (len(df_motion)/freq)/60, len(df_motion)) # minutes

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))
labels = ['X', 'Y', 'Z']
colors = ['tab:orange','tab:blue','tab:green']

for anr in range(3):
    ax[anr].plot(time, filtnorm[anr], label=labels[anr], linewidth=0.5, color=colors[anr], alpha=0.5)
    ax[anr].legend(loc='lower right', fontsize=14)
ax[0].set_title('X - Lateral - side to side')
ax[1].set_title('Y - Vertical - up down')
ax[2].set_title('Z - Frontal - forwards backwards')

fig.subplots_adjust(hspace=0.5)

### Plot just one minute of the filtered and normalized data for comparison

In [ ]:
pt = 14
kho_aud = khoparzi_audience[pt]
kho_filtnorm = khoparzi_normalized[pt]

# extract 1-min
dance_excerpt = kho_aud.sort_index().loc['2020-07-10 14:45' : '2020-07-10 14:46', :] # Real times: '2020-07-10 14:36' : '2020-07-10 15:04'


dance_excerpt_filtnorm = []
for anr in range(3):
    ax = kho_filtnorm[anr]
    excerpt = ax[1:7200] # specifies the indices for specifying the time.
    dance_excerpt_filtnorm.append(excerpt)


In [ ]:
# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(dance_excerpt.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')

In [ ]:
# create x-axis
time = np.linspace(0, (len(dance_excerpt)/freq)/60, len(dance_excerpt)) # minutes

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))
labels = ['X', 'Y', 'Z']
colors = ['tab:orange','tab:blue','tab:green']

for anr in range(3):
    ax[anr].plot(time, dance_excerpt_filtnorm[anr], label=labels[anr], linewidth=0.5, color=colors[anr], alpha=0.5)
    ax[anr].legend(loc='lower right', fontsize=14)
ax[0].set_title('X - Lateral - side to side')
ax[1].set_title('Y - Vertical - up down')
ax[2].set_title('Z - Frontal - forwards backwards')

fig.subplots_adjust(hspace=0.5)

## 5. Feature Extraction
### 5.1. Quantity of Motion (QoM)

In [ ]:
def qom(time, x, y, z):
    qom=[]
    for i in range(len(time)-1):
        id1 = sum((x[i],y[i],z[i]))
        id2 = sum((x[i+1],y[i+1],z[i+1]))
        diff = abs(id2-id1)
        qom.append(diff)
    return qom

#Note that this function can be written more elegantly

In [ ]:
#Filter QoM for a better representation
from scipy.signal import savgol_filter as savgol

#normalized QoM:
qom_n = normalize(qom(time,filtnorm[0],filtnorm[1],filtnorm[2])) 

# normalized (filtered) QoM trend:
qom_filt_order = 1
qom_win = 1999 #reduce for more resolution
qom_fn = normalize(savgol(qom(time,filtnorm[0],filtnorm[1],filtnorm[2]), qom_win, qom_filt_order)) 
                                                                

def plot_qoms(qom_n, qom_fn, fontsize=15):

    plt.figure(figsize=(24,7))
    plt.plot(time[:-1], qom_n, alpha=0.2, color='r', label='QoM')
    plt.plot(time[:-1], qom_fn, alpha=0.7, color='b', label='QoM Trend')
    plt.xticks(fontsize=15)
    plt.xlabel('Time (m)',fontsize=fontsize)
    plt.yticks(fontsize=15)
    plt.ylabel('Amplitude',fontsize=fontsize)
    plt.legend(loc='upper right', fontsize=fontsize)

In [ ]:
plot_qoms(qom_n, qom_fn)

In [ ]:
# Planar acceleration

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,7))

ax[0].plot(filtnorm[0], filtnorm[2], color='green', alpha=0.3, label='XZ')
ax[1].plot(filtnorm[0], filtnorm[1], color='red', alpha=0.3, label='XY')
ax[2].plot(filtnorm[1], filtnorm[2], color='blue', alpha=0.3, label='YZ')

[ax[n].legend(loc='upper right') for n in range(3)]

fig.subplots_adjust(wspace=.5)

### 5.2. Motion peaks

In [ ]:
import sensormotion as sm

def avg(axes_list):
    return np.sum([axes_list[0], axes_list[1], axes_list[2]], axis=0) / 3

peak_times, peak_values = sm.peak.find_peaks(time=time, signal=avg(filtnorm), peak_type='valley',
                                             min_val=0.6, min_dist=freq//2, 
                                             plot=True, fig_size=(24,5))

### 5.3. Periodograms

In [ ]:
#Note: Also try zero-crossings and autocorrelation

In [ ]:
from scipy import signal

periodograms=[]
for i in range(3):
    f, Pxx = signal.periodogram(filtnorm[i], fs=freq, window='hanning', scaling='spectrum')
    periodograms.append((f,Pxx))

In [ ]:
def plot_periods(periodogramz):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16,7))
    colors = ['r','g', 'b']
    for idx,(f,Pxx) in enumerate(periodogramz):
        
        ax[idx].plot(f, Pxx, color=colors[idx], alpha=0.4)
        ax[idx].set_yscale('log') #Comment out for linear scale
        ax[idx].set_xlabel('Frequency (Hz)')
        ax[0].set_ylabel('Spectrum Amplitude')
          
    fig.subplots_adjust(wspace=.5)

In [ ]:
plot_periods(periodograms)

In [ ]:
def top_periods(f, Pxx):
    '''f=[sample frequencies]; Pxx=[power spectrum of x]'''
    tops={}
    top_fq_indices = np.flip(np.argsort(Pxx), 0)[0:3]
    freqs = f[top_fq_indices]
    power = Pxx[top_fq_indices]
    periods = 1 / np.array(freqs)
    
    for i in range(1,4):
        tops['Period_{} (secs)'.format(i)] = round(periods[i-1], 2)
        tops['Freq_{} (Hz)'.format(i)] = round(freqs[i-1], 3)
        tops['Tempo_{} (BPM)'.format(i)] = round(freqs[i-1]*60, 2)
        tops['Power_{} (A)'.format(i)] = format(power[i-1], 'f')
        
    return tops

In [ ]:
x_periods = top_periods(periodograms[0][0], periodograms[0][1])
y_periods = top_periods(periodograms[1][0], periodograms[1][1])
z_periods = top_periods(periodograms[2][0], periodograms[2][1])

In [ ]:
#x_periods
#y_periods
z_periods

## 6. Audio

In [ ]:
import librosa
from librosa import display
import IPython.display as ipd
path = 'C:/Users/danasw/Documents/PhD/Research/MusicLab_Lockdown/Files_for_Research_Drive/Raw_Data/Audio/'

## Get sampling rate

In [ ]:
from scipy.io.wavfile import read as read_wav
import os
os.chdir(path) # change to the file directory
rb_sampling_rate, data=read_wav( 'Renick_Bell.wav') # enter your filename
print('Renick Bell sr: '+str(rb_sampling_rate))
k_sampling_rate, data=read_wav( 'Khoparzi-Audio.wav') # enter your filename
print('Khoparzi sr: '+str(k_sampling_rate))

In [ ]:
renick_audio, sr = librosa.load(os.path.join(path, 'Renick_Bell.wav'), sr=44100) # include argument `duration=duration*60` if you wanr to cut the sound based on the motion data
ipd.Audio(renick_audio, rate=sr)

In [ ]:
khoparzi_audio, sr = librosa.load(os.path.join(path, 'Khoparzi-Audio.wav'), sr=48000) # include argument `duration=duration*60` if you wanr to cut the sound based on the motion data
ipd.Audio(khoparzi_audio, rate=sr)

### 6.1. Source separation
### 6.2. Onset detection
### 6.3. Tempo tracking

In [ ]:
# Notes: Here I detect onset based on the decomposed percussive components. BUT, also use non-decomposed signal. 
# The outcome differs depending on the musical content
# In addition, dynamic tempo tracking is a better option for rhythmically non-conventional musics

In [ ]:
def onset_tempo(sound, sr=44100, hop_length=512, n_fft=2048, Normalize=True):
    perc = librosa.effects.percussive(sound) #6.1.
    onset_strength = librosa.onset.onset_strength(y=perc, sr=sr, hop_length=hop_length) #6.2.
    onset_strength_n = normalize(onset_strength)
    time_s = librosa.times_like(onset_strength, sr=sr, hop_length=hop_length, n_fft=n_fft)
    tempo = librosa.beat.tempo(onset_envelope=onset_strength, sr=sr) #6.3.
    if Normalize:
        return time_s, onset_strength_n, tempo
    else:
        return time_s, onset_strength, tempo

In [ ]:
audio_features_renick = onset_tempo(renick_audio)

In [ ]:
audio_features_khoparzi = onset_tempo(khoparzi_audio, sr = 48000)

In [ ]:
print('Renick Tempo: %.i BPM'%(audio_features[2][0]))

In [ ]:
print('Khoparzi Tempo: %.i BPM'%(audio_features[2][0]))

In [ ]:
#Plot all together

In [ ]:
peak_values_n = normalize(peak_values)

fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(16,7))

ax[0].plot(audio_features[0], audio_features[1], alpha=0.4, linewidth=2.5, label='Audio onset strength', color='blue')

ax[1].plot(peak_times, peak_values_n, alpha=0.4, linewidth=2.5, label='ACC peaks', color='red')

ax[2].plot(time[:-1], qom_fn, alpha=0.4, linewidth=2.5, label='QoM trend', color='green')

[ax[n].grid(True) for n in range(3)]
[ax[n].legend(loc='lower center') for n in range(3)]

## 7. Geo-location

In [ ]:
df_geo = pd.read_csv(geo_dat,index_col=0)
df_geo.head()

In [ ]:
from mpl_toolkits.basemap import Basemap
from geopy.geocoders import Nominatim

def get_address(lat, long, api='geoapiExercises'):
    geolocator = Nominatim(user_agent=api)
    location = geolocator.reverse(str(lat)+ ', ' + str(long))
    return location.address.split(', ')

def get_coordinates(place, api='geoapiExercises', timeout=5):
    geolocator = Nominatim(user_agent=api)
    location = geolocator.geocode(place, timeout=timeout)
    if not location:
        return None, None
    return location.latitude, location.longitude

def plot_map(latitude, longitude, figsize=(10,10), markersize=12, fontsize=20, verbose=False):
    address = get_address(latitude, longitude)
    if verbose:
        print(address)
    address = address[-3:]
    city_lat = get_coordinates(address[0])[0]
    city_lon = get_coordinates(address[0])[1]
    
    fig = plt.figure(figsize=figsize)
    m = Basemap(projection='lcc', resolution=None, 
                width=8E6, height=8E6,
                lat_0 = city_lat, lon_0 = city_lon)
    m.etopo(scale=1, alpha=0.5)
    x, y = m(longitude, latitude)
    plt.plot(x, y, 'ok', markersize=markersize)
    plt.text(x, y, address[0], fontsize=fontsize)

In [ ]:
plot_map(df_geo.latitude.values[0], df_geo.longitude.values[0], verbose=True)